In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import LinearRegression
from __future__ import division

import warnings
warnings.filterwarnings('ignore')

Merge files and subset

### Data Dictionary

- SERIALNO: household id
- FES: married, heterosexual couples are 1, 2, 3 or 4
- SSMC: same-sex marriage flag
- WAGP: wage
- SEX: male=1, female=2
- SCHL: level of education (16=HS, 21=Bachelor's, 22=Masters, 24=PhD)
- RAC1P: race (1=white alone)
- HHL: household language (1=English only)
- AGEP: age
- CIT: citizenship status (5=non citizen)
- NOC: number of children in household
- COW: class of worker (for profit / government / self-employed etc)
- DIS: with disability (1=with)
- WKHP: weekly hours worked
- ST: state
- PUMA: area

In [2]:
# variables of interest
keep_vars = ['SERIALNO', 'SSMC', 'HHL', 'NOC', 'gay', 'ST', 'PUMA', 'AGEP', 'CIT', 'COW',\
             'SCHL', 'SEX', 'WAGP', 'WKHP', 'DIS']

In [3]:
def munge(list_files, keep_vars):
    ''' joins dataframes, and subsets to required features'''
    df = pd.read_csv(list_files[0])
    print list_files[0], "read"
    df = df[keep_vars]
    cols = df.columns
    for i in list_files[1:]:
        print i, "read"
        temp = pd.read_csv(i)
        temp = temp[keep_vars]
        df = pd.concat([df, temp])
    return df

In [4]:
files = ['sub14.csv', 'sub15.csv', 'sub16.csv'] #files to merge

In [5]:
df = munge(files, keep_vars) # merge files into df

sub14.csv read
sub15.csv read
sub16.csv read


In [6]:
df.to_csv('sub.csv', index=False) # save out merged data

In [7]:
df.shape

(3651076, 15)

In [32]:
len(df.loc[df['gay']==1])

27874

In [31]:
pd.pivot_table(df, index='gay', columns = 'SEX', values='WAGP') # wages by sex and ssmc

SEX,0,1
gay,,
0,25193.370167,49872.439039
1,39894.756335,52392.715026


### Feature cleaning and engineering

In [8]:
df['eng'] = df['HHL'].apply(lambda x: 1 if x==1 else 0) # 1 if household language is english, else 0

In [9]:
df['SEX'] = df['SEX'].apply(lambda x: 0 if x==2 else 1) # dummy encode sex, 1 for male, 0 for female

In [10]:
df = pd.get_dummies(df, columns=['COW', 'CIT']) #get dummies for class of worker and citizenship

In [11]:
# function to convert working hours to float
def clean_wkhp(x):
    try:
        return float(x)
    except:
        return 0

In [12]:
df['WKHP'] = df['WKHP'].apply(clean_wkhp)
df['WKHP'] = df['WKHP'].fillna(0)

In [13]:
df['WAGP_log'] = df['WAGP'].apply(lambda x: 0 if x ==0 else np.log(x)) # log wages

### Country level model

In [14]:
X_cols = [ u'NOC', u'gay', u'AGEP',
      u'SCHL', u'SEX',u'WKHP',u'eng', u'COW_1.0',
       u'COW_2.0', u'COW_3.0', u'COW_4.0', u'COW_5.0', u'COW_6.0', u'COW_7.0',
       u'COW_8.0', u'COW_9.0', u'CIT_1', u'CIT_2', u'CIT_3', u'CIT_4',
       u'CIT_5'] 
# use number of children, age, education, sex, working hours, household language, citizenship status
# and class of worker


In [15]:
y = df['WAGP_log'] # predicting log wages
X = df[X_cols]

In [16]:
lr = LinearRegression()
lr.fit(X, y)
lr.score(X, y) # check R2

0.85790595417536997

In [17]:
coef_df = pd.DataFrame(zip(X.columns, lr.coef_), columns=['Feature', 'Coef'])
coef_df.sort_values('Coef')

,Feature,Coef
12,COW_6.0,-3.963862
4,SEX,-0.158085
15,COW_9.0,-0.103223
20,CIT_5,-0.045237
17,CIT_2,-0.044377
16,CIT_1,-0.041766
2,AGEP,-0.003384
6,eng,0.015234
18,CIT_3,0.034649
0,NOC,0.037564


Notables:
- women make 16% less
- age association is tiny
- more hours, more money
- more children, higher wages
- people in same sex marriage make 9% more

### State Level Models

In [18]:
states = df['ST'].unique() # list of all states

In [19]:
# loop through all states and build model for each one.  store coef for SSM members
scores = []
total_num = []
ng = []
coefs = []
for st in states:
    sub = df.loc[df['ST']==st] #subset by state
    total_num.append(len(sub)) #store total number of observations
    ng.append(len(sub.loc[sub['gay']==1])) #store number of same sex marriage members
    y = sub['WAGP_log'] #build models
    X = sub[X_cols]
    lr.fit(X, y)
    scores.append(lr.score(X, y)) #store R2
    coefs.append(lr.coef_[1]) #store coef for ssm

In [20]:
results = pd.DataFrame(zip(states, scores, ng, total_num, coefs), \
             columns = ['state', 'score', 'num_ss', 'num', 'coef'])
results.sort_values('num_ss').head(10)

,state,score,num_ss,num,coef
1,2,0.785049,24,6320,0.576537
41,46,0.811739,30,10902,-0.384862
50,56,0.816888,38,7570,-0.351493
26,30,0.809661,44,13308,-0.157988
34,38,0.794704,46,10386,0.422905
12,16,0.824889,82,20480,0.033135
48,54,0.886204,106,22438,0.082918
27,31,0.824483,106,24766,-0.098775
45,50,0.823911,108,8466,-0.161541
24,28,0.866555,124,30908,-0.006354


In [21]:
# for any state with fewer than 100 ssmc, set data to na
results.loc[results['num_ss']<100, 'coef'] = np.nan
#write to file for mapping
results.to_csv('stateresults.csv', index=False)

In [147]:
results.sort_values('coef').head()

,state,score,num_ss,num,coef
45,50,0.823911,108,8466,-0.161541
2,4,0.874090,530,74592,-0.129880
27,31,0.824483,106,24766,-0.098775
44,49,0.847666,202,35528,-0.097875
14,18,0.864321,550,78924,-0.096111


In [149]:
results.sort_values('coef').tail(15)

,state,score,num_ss,num,coef
28,32,0.864482,280,29508,0.126330
23,27,0.832277,472,73364,0.145197
22,26,0.867468,532,121066,0.160955
31,35,0.863057,198,20248,0.179473
49,55,0.834925,342,78702,0.188998
40,45,0.869945,322,57232,0.232621
36,40,0.862030,250,42764,0.240558
39,44,0.857859,136,11646,0.295015
3,5,0.867417,160,35778,0.309194
1,2,0.785049,24,6320,NaN


- coefficients range from -0.16 in Vermont (unexpected!) to 0.3 in Arkansas (also unexpected!)
- indicates that being out / in a same-sex marriage might be only accessible to better paid people in traditionally conservative states

### GWR with PUMA

In [22]:
centroids = pd.read_csv('puma_centroids.csv') # import file with centroids
centroids.head()

,STATEFP10,PUMACE10,GEOID10,lon,lat
0,23,700,2300700,-70.059525,43.923567
1,23,100,2300100,-68.319673,46.190257
2,23,900,2300900,-70.383773,43.656080
3,23,800,2300800,-70.715292,43.585686
4,23,200,2300200,-69.924391,45.378369


In [24]:
# create puma in data to match centroids
def puma(row):
    return '{:02d}{:05d}'.format(int(row['ST']), int(row['PUMA']))

In [25]:
df['puma_full'] = df.apply(puma, axis=1)
df.head()

,SERIALNO,SSMC,HHL,NOC,gay,ST,PUMA,AGEP,SCHL,SEX,...,COW_7.0,COW_8.0,COW_9.0,CIT_1,CIT_2,CIT_3,CIT_4,CIT_5,WAGP_log,puma_full
0,171807,2.0,1.0,0.0,1,1,2702,46,19.0,0,...,0,0,0,1,0,0,0,0,9.740969,0102702
1,171807,2.0,1.0,0.0,1,1,2702,50,18.0,0,...,0,0,0,1,0,0,0,0,9.615805,0102702
2,240124,2.0,1.0,0.0,1,1,1100,49,18.0,0,...,0,0,0,1,0,0,0,0,0.000000,0101100
3,240124,2.0,1.0,0.0,1,1,1100,56,18.0,0,...,0,0,0,1,0,0,0,0,0.000000,0101100
4,241520,2.0,3.0,0.0,1,1,2500,56,16.0,0,...,0,0,0,1,0,0,0,0,9.798127,0102500


In [26]:
df['puma_full'] = df['puma_full'].astype(int) # convet to ints to match

In [27]:
puma_merge = df.merge(centroids, left_on='puma_full', right_on='GEOID10', how='inner')# merge data sources

In [28]:
print len(puma_merge)
puma_merge.head()
# have 3.6mm observations

3651076


,SERIALNO,SSMC,HHL,NOC,gay,ST,PUMA,AGEP,SCHL,SEX,...,CIT_3,CIT_4,CIT_5,WAGP_log,puma_full,STATEFP10,PUMACE10,GEOID10,lon,lat
0,171807,2.0,1.0,0.0,1,1,2702,46,19.0,0,...,0,0,0,9.740969,102702,1,2702,102702,-88.167205,30.691743
1,171807,2.0,1.0,0.0,1,1,2702,50,18.0,0,...,0,0,0,9.615805,102702,1,2702,102702,-88.167205,30.691743
2,417636,2.0,1.0,0.0,1,1,2702,78,18.0,0,...,0,0,0,0.000000,102702,1,2702,102702,-88.167205,30.691743
3,417636,2.0,1.0,0.0,1,1,2702,79,18.0,0,...,0,0,0,0.000000,102702,1,2702,102702,-88.167205,30.691743
4,15516,0.0,1.0,0.0,0,1,2702,73,22.0,0,...,0,0,0,4.787492,102702,1,2702,102702,-88.167205,30.691743


In [29]:
# remove alaska, hawaii and puerto rico since they are geographically separated
sub_merge = puma_merge[~puma_merge['ST'].isin([2,15,72])]

Subsetting the data
- 3.6mm is probably prohibively large for GWR
- I subset the data by keeping all SSMC records (since there are only 26k), and randomly sampling opposite-sex marriage records to match the number by PUMS area

In [35]:
pums = list(set(sub_merge['puma_full'])) # list of all pumas

In [36]:
sub_sampled = pd.DataFrame()
for i, p in enumerate(pums):
    if i%5==0:
        print i
    temp = sub_merge[sub_merge['puma_full']==p]
    temp_g = temp[temp['gay']==1]
    num = len(temp_g)
    temp_h = temp[temp['gay']==0].sample(n=num) # sample from non SSMC to match numbers
    sub_sampled = pd.concat([sub_sampled, temp_g, temp_h])

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
12

In [39]:
len(sub_sampled)

55432

GWR 
- using gaussian kernel to calculate weights for regression
- using statsmodel WLS

In [81]:
from gaussian import Gaussian
import statsmodels.api as sm

In [87]:
#unique puma areas and coords of centroids
unique_locations = sub_sampled[['puma_full', 'lat', 'lon']].drop_duplicates() 
pumas = unique_locations['puma_full'].values
locations = unique_locations[['lat', 'lon']].values

In [97]:
#subset data

y = sub_sampled['WAGP_log'].values #target

#feature columns
X_cols = [ u'NOC', u'gay', u'AGEP',
      u'SCHL', u'SEX',u'WKHP',u'eng', u'COW_1.0',
       u'COW_2.0', u'COW_3.0', u'COW_4.0', u'COW_5.0', u'COW_6.0', u'COW_7.0',
       u'COW_8.0', u'COW_9.0', u'CIT_1', u'CIT_2', u'CIT_3', u'CIT_4',
       u'CIT_5']

x = sub_sampled[X_cols].values #features




In [101]:
sub_w = sub_sampled[X_cols+['puma_full', 'WAGP_log']] # include area in data

In [120]:
# bandwidth 1
# loop through each area and build model, store parmas
params = []
for i, p in enumerate(pumas): 
    w = Gaussian.kernel(locations, locations[i], 1)[0] # calculate the weights
    ws = []
    for area in sub_w['puma_full']: # for each record, find approprate weights
        ws.append(w[np.where(pumas==area)])
    x = sub_sampled[X_cols].values
    y = sub_sampled['WAGP_log'].values
    res = sm.WLS(y, x, ws) # calculate wls
    fres = res.fit()
    params.append(fres.params)
    if i%100==0:
        print i


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [129]:
# subset to just coefficient for ssmc
gwr_coefs = pd.DataFrame(zip(pumas, [i[1] for i in params]), columns = ['puma', 'gay_coef'])

In [130]:
gwr_coefs.to_csv('gwr_results.csv', index=False) # save for mapping

In [133]:
# bandwidth 3
params = []
for i, p in enumerate(pumas):
    w = Gaussian.kernel(locations, locations[i], 3)[0]
    ws = []
    for area in sub_w['puma_full']:
        ws.append(w[np.where(pumas==area)])
    x = sub_sampled[X_cols].values
    y = sub_sampled['WAGP_log'].values
    res = sm.WLS(y, x, ws)
    fres = res.fit()
    params.append(fres.params)
    if i%100==0:
        print i
gwr_coefs = pd.DataFrame(zip(pumas, [i[1] for i in params]), columns = ['puma', 'gay_coef'])
gwr_coefs.to_csv('gwr_results_3.csv', index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [144]:
# susbet to men and women for separate models
women = sub_w[sub_w['SEX']==1]
men = sub_w[sub_w['SEX']==0]

In [141]:
X_cols.remove('SEX')

In [143]:
# bandwidth 3, women
params = []
for i, p in enumerate(pumas):
    w = Gaussian.kernel(locations, locations[i], 3)[0]
    ws = []
    for area in women['puma_full']:
        ws.append(w[np.where(pumas==area)])
    x = women[X_cols].values
    y = women['WAGP_log'].values
    res = sm.WLS(y, x, ws)
    fres = res.fit()
    params.append(fres.params)
    if i%100==0:
        print i
gwr_coefs = pd.DataFrame(zip(pumas, [i[1] for i in params]), columns = ['puma', 'gay_coef'])
gwr_coefs.to_csv('gwr_results_women.csv', index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [145]:
# bandwidth 3, men
params = []
for i, p in enumerate(pumas):
    w = Gaussian.kernel(locations, locations[i], 3)[0]
    ws = []
    for area in men['puma_full']:
        ws.append(w[np.where(pumas==area)])
    x = men[X_cols].values
    y = men['WAGP_log'].values
    res = sm.WLS(y, x, ws)
    fres = res.fit()
    params.append(fres.params)
    if i%100==0:
        print i
gwr_coefs = pd.DataFrame(zip(pumas, [i[1] for i in params]), columns = ['puma', 'gay_coef'])
gwr_coefs.to_csv('gwr_results_men.csv', index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
